# Homework 3

Please submit the solution to gradescope by 11:59 PM, Oct 10, Thursday.

**Name**: Ivy Nangalia 

**PID**: 730670491


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 1. DataFrame with MultiIndex (12 points)

You are given the following MultiIndexed DataFrame temperature_data which contains monthly average temperatures (in °C) for different cities over two years. 

- 1.1. Display the temperature data for the year 2019.
- 1.2. Display the temperature data for 'New York' city across all years.
- 1.3. Display the temperature data for the month of 'Jan' across all cities and years.
- 1.4. Display temperature data for 'Los Angeles' city for the year 2020.
- 1.5. Display the temperature data for the 'Chicago' and 'Houston' cities for the month of 'Feb' in the year 2019.
- 1.6. Display the temperature data for the months 'Mar' and 'Apr' for all cities in the year 2020.

In [3]:
index = pd.MultiIndex.from_product(
    [['2019', '2020'], ['New York', 'Los Angeles', 'Chicago', 'Houston'], ['Jan', 'Feb', 'Mar', 'Apr']],
    names=['Year', 'City', 'Month']
)

np.random.seed(42)
data = np.random.uniform(-10, 35, size=(32, 1))

temperature_data = pd.DataFrame(data, index=index, columns=['Avg_Temperature'])
temperature_data.sort_index(inplace=True)
temperature_data

Avg_Temperature
Year City        Month                 
2019 Chicago     Apr          33.645943
                 Feb          21.863266
                 Jan          17.050176
                 Mar          -9.073698
     Houston     Apr          -1.746797
                 Feb          -0.444740
                 Jan          27.459919
                 Mar          -1.817876
     Los Angeles Apr          28.977927
                 Feb          -2.980247
                 Jan          -2.979161
                 Mar          -7.386237
     New York    Apr          16.939632
                 Feb          32.782144
                 Jan           6.854305
                 Mar          22.939727
2020 Chicago     Apr          13.140550
                 Feb          25.332918
                 Jan          10.523149
                 Mar          -1.014680
     Houston     Apr          -2.326414
                 Feb          -7.909731
                 Jan          16.658656
                 Mar          17.339518
     Los Angeles Apr           6.486283
                 Feb          -3.722776
                 Jan          17.533380
                 Mar           3.146509
     New York    Apr           3.105311
                 Feb          13.614039
                 Jan           3.690901
                 Mar           9.437526

In [17]:
#1.1
temperature_data.loc[:"2019"]

Avg_Temperature
Year City        Month                 
2019 Chicago     Apr          33.645943
                 Feb          21.863266
                 Jan          17.050176
                 Mar          -9.073698
     Houston     Apr          -1.746797
                 Feb          -0.444740
                 Jan          27.459919
                 Mar          -1.817876
     Los Angeles Apr          28.977927
                 Feb          -2.980247
                 Jan          -2.979161
                 Mar          -7.386237
     New York    Apr          16.939632
                 Feb          32.782144
                 Jan           6.854305
                 Mar          22.939727

In [18]:
#1.2
temperature_data.loc[:, 'New York', :]


Avg_Temperature
Year Month                 
2019 Apr          16.939632
     Feb          32.782144
     Jan           6.854305
     Mar          22.939727
2020 Apr           3.105311
     Feb          13.614039
     Jan           3.690901
     Mar           9.437526

In [19]:
#1.3
temperature_data.loc[:, :, "Jan"]

Avg_Temperature
Year City                        
2019 Chicago            17.050176
     Houston            27.459919
     Los Angeles        -2.979161
     New York            6.854305
2020 Chicago            10.523149
     Houston            16.658656
     Los Angeles        17.533380
     New York            3.690901

In [21]:
#1.4
temperature_data.loc["2020", "Los Angeles", :]

,Avg_Temperature
Month,
Apr,6.486283
Feb,-3.722776
Jan,17.533380
Mar,3.146509


In [28]:
#1.5
temperature_data.loc[("2019", ["Chicago", "Houston"], "Feb")]

Avg_Temperature
Year City    Month                 
2019 Chicago Feb          21.863266
     Houston Feb          -0.444740

In [35]:
#1.6
cities = temperature_data.City.unique()
temperature_data.loc[("2020", cities, ["Feb", "April"])]

AttributeError: 'DataFrame' object has no attribute 'City'

# 2. Data Merge and Join (68 points)

2.1 Load `product_data.csv` and `sales_data.csv` as DataFrames. (2 points) (In the columns, both `ProductID` and `ID_Product` refers to the product ID.)

2.2 Calculate Total Quantity Sold per Product, i.e., create a pd.Series called `product_sales_df` that contains the sales data for each product and display it. The index of the pd.Series is the product name. (10 points)

2.3 Perform a join to merge `product_sales_df` with the product data on ProductID to include all products. Why does the merged table have missing data? (10 points)

2.4 Fill the missing data in the table from 2.3 with 0. Calculate total and average quantities sold for each category, i.e., create a new DataFrame that contains three columns: `Category`, `TotalQuantitySold`, and `AverageQuantitySold`. (5 points)

2.5  Calculate the total revenue per product and identifying the product that generated the highest revenue. The revenue equals the `sales` $\times$ `unit price`. (10 points)


2.6 Identify Top Products per Category Based on Total Quantity Sold, i.e., create a table with three columns. The first column is the `category` name, the second column is the name of the product with the highest sales in that category, and the third column is the sales for that product. (10 points)

2.7 Create a table with two layers of row index, showing the total revenue for each category in each month. The first row index is the category and the second row index is the month. (5 points)

2.8 Creat a histogram to show the total quantity sold per category. Properly display the x-axis, y-axis and the title of the plot. (6 points)

2.9 Create a pivot table of the total quantity and the average quantity sold by product and month. The row index should be the product name, and the column index has two layers: The upper layer is the month, and the lower layer is the `TotalQuantity` and `AverageQuantity`. For example, the first column index in the upper layer is `2023-01`. Under `2023-01`, there are two columns named `AverageQuantity` (mean) and `TotalQuantity` (sum). (10 points)

Hint: You can use [swaplevel](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.swaplevel.html) function in pd.DataFrame to change the index layer. For example: `pivot_table = pivot_table.swaplevel(0, 1, axis=1)` can swap the index layer for the pivot table

In [39]:
# 2.1
product_data = pd.read_csv("product_data.csv")
sales_data = pd.read_csv("sales_data.csv")
product_data

,ID_Product,ProductName,Category,UnitPrice
0,P101,Samsung Galaxy S21,Smartphones,799.99
1,P102,Apple iPhone 13,Smartphones,999.99
2,P103,HP Spectre x360,Laptops,1199.99
3,P104,Dell XPS 13,Laptops,1099.99
4,P105,Apple MacBook Air,Laptops,999.99
5,P106,LG OLED55CXPUA,Televisions,1499.99
6,P107,Sony WH-1000XM4,Headphones,349.99
7,P108,Bose QuietComfort 35 II,Headphones,299.99
8,P109,Canon EOS R5,Cameras,3899.99
9,P110,Nikon Z7 II,Cameras,2999.99


In [38]:
#2.2
product_sales_df: pd.Series
for product in product_data["ID_Product"]:
    for id in sales_data["ProductID"]:
        if product == id:
            product_sales_df[product] += 

,TransactionID,CustomerID,ProductID,Quantity,SaleDate,Month
0,13,C1010,P113,2,2023-01-17,2023-01
1,37,C1005,P119,2,2023-02-10,2023-02
2,33,C1001,P115,1,2023-02-06,2023-02
3,19,C1007,P119,3,2023-01-23,2023-01
4,21,C1009,P101,1,2023-01-25,2023-01
5,38,C1006,P120,1,2023-02-11,2023-02
6,31,C1009,P113,1,2023-02-04,2023-02
7,3,C1001,P103,1,2023-01-07,2023-01
8,26,C1004,P106,1,2023-01-30,2023-01
9,29,C1007,P111,1,2023-02-02,2023-02


# 3. Webscrapping: Collect the Emails of all faculty in the STOR department  (20 points)

Use web scrapping technique to collect the email information of all faculty in the UNC STOR department from the webpage: [https://stor.unc.edu/people/faculty/](https://stor.unc.edu/people/faculty/) 


3.1. Create a pd.DataFrame that contains three columns: `faculty_name`, `title` and `email_address`. (10 points)
For example, one row of the dataframe is `Liu, Mo`, `Assistant Professor`, and `Mo_Liu@unc.edu`.



3.2. Notice that in the DataFrame in 3.1, the last name and first name of each person are stored in the same column and separated by `,`. Create two new columns `First_Name` and `Last_Name` that contains the first name and last name seperately. (10 points)